In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [91]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [93]:
# reading data
df = pd.read_csv('cars_data_clean.csv')
# df.head()

In [95]:
df.columns
excluded = ['usedCarSkuId', 'images', 'loc',
            'imgCount', 'dvn', 'City', 'Drive Type',
            'state', 'Compression Ratio', 'Alloy Wheel Size', 
            'Ground Clearance Unladen', 'Stroke', 'Bore', 
            'Length', 'Width', 'Height', 
            'Valve Configuration','Max Torque At',
            'Max Power At','Fuel Suppy System', 'discountValue', 'top_features' ]

In [97]:
df.drop(columns=excluded,inplace=True)
# df.head()
# mainDf.info()

In [99]:
rename_dict = {
     'Color': 'color',
    'Engine Type': 'engine_type', 'No of Cylinder': 'no_of_cylinder',
    'Valves per Cylinder': 'valves_per_cylinder', 'Turbo Charger': 'turbo_charger',
    'Super Charger': 'super_charger', 'Wheel Base': 'wheel_base',
    'Front Tread': 'front_tread', 'Rear Tread': 'rear_tread',
    'Kerb Weight': 'kerb_weight', 'Gross Weight': 'gross_weight',
    'Gear Box': 'gear_box', 'Seats': 'seats', 'Steering Type': 'steering_type',
    'Turning Radius': 'turning_radius', 'Front Brake Type': 'front_brake_type',
    'Rear Brake Type': 'rear_brake_type', 'Top Speed': 'top_speed',
    'Acceleration': 'acceleration', 'Tyre Type': 'tyre_type', 'Doors': 'doors',
    'Cargo Volume': 'cargo_volume', 'model_type_new': 'model_type_new',
    'exterior_color': 'exterior_color', 'owner_type': 'owner_type',
    'Max Power Delivered': 'max_power_delivered', 'Max Torque Delivered': 'max_torque_delivered'
}

df = df.rename(columns=rename_dict)

In [103]:
# mainDf.describe()
missing_values = df.isna().sum()
missing_values = missing_values[missing_values > 0]  # Filter out columns with 0 missing values

print(missing_values)

body                       19
color                     160
engine_type               868
no_of_cylinder            143
valves_per_cylinder       228
wheel_base                556
front_tread             15874
rear_tread              15875
kerb_weight             13139
gross_weight            20890
gear_box                  471
seats                      18
steering_type             808
turning_radius           5848
front_brake_type          327
rear_brake_type           326
top_speed               15352
acceleration            13988
tyre_type                 257
doors                      44
cargo_volume             6732
exterior_color            123
max_power_delivered       231
max_torque_delivered      231
dtype: int64


In [105]:
df['body'].describe()
df['body'].unique()

array(['hatchback', 'sedan', 'muv', 'minivans', nan, 'pickup trucks',
       'suv', 'luxury vehicles', 'convertibles', 'coupe', 'wagon',
       'hybrids'], dtype=object)

## filling Nan values of categorical values
### Approaches
    1. Removing na row/columns
    2. Fill missing categorical values with 'Unknown' (new category)
    3. Fill with Mean/Median/Mode
    4. Forward or Backward Fill (method='ffill/bfill')
    5. Fill with SimpleImputer approach

In [107]:
df['body'].fillna(df['body'].mode()[0], inplace=True)
naColumns = [col for col in df.columns if df[col].isna().sum() > 0]
naColumns

['color',
 'engine_type',
 'no_of_cylinder',
 'valves_per_cylinder',
 'wheel_base',
 'front_tread',
 'rear_tread',
 'kerb_weight',
 'gross_weight',
 'gear_box',
 'seats',
 'steering_type',
 'turning_radius',
 'front_brake_type',
 'rear_brake_type',
 'top_speed',
 'acceleration',
 'tyre_type',
 'doors',
 'cargo_volume',
 'exterior_color',
 'max_power_delivered',
 'max_torque_delivered']

In [109]:
# filling Nan suing simpleimputer
from sklearn.impute import SimpleImputer
cat_imputer = SimpleImputer(strategy='most_frequent')
df[naColumns] = cat_imputer.fit_transform(df[naColumns])
df.shape

(37813, 44)

In [67]:
df.isna().sum()

myear                   0
body                    0
transmission            0
fuel                    0
km                      0
ip                      0
threesixty              0
oem                     0
model                   0
variant                 0
listed_price            0
utype                   0
carType                 0
comfort_features        0
interior_features       0
exterior_features       0
safety_features         0
Color                   0
Engine Type             0
No of Cylinder          0
Valves per Cylinder     0
Turbo Charger           0
Super Charger           0
Wheel Base              0
Front Tread             0
Rear Tread              0
Kerb Weight             0
Gross Weight            0
Gear Box                0
Seats                   0
Steering Type           0
Turning Radius          0
Front Brake Type        0
Rear Brake Type         0
Top Speed               0
Acceleration            0
Tyre Type               0
Doors                   0
Cargo Volume

In [111]:
df.describe()

,myear,km,ip,listed_price
count,37813.000000,3.781300e+04,37813.000000,3.781300e+04
mean,2015.569513,6.240930e+04,0.010208,7.999868e+05
std,3.779089,5.847216e+04,0.100520,3.043045e+06
min,1983.000000,1.010000e+02,0.000000,1.196300e+04
25%,2013.000000,3.173900e+04,0.000000,3.200000e+05
50%,2016.000000,5.672600e+04,0.000000,5.250000e+05
75%,2018.000000,8.331000e+04,0.000000,8.552720e+05
max,2023.000000,6.300000e+06,1.000000,5.500006e+08


In [115]:
# data type conversion
df = df.rename(columns={'listed_price': 'selling_price'})

In [117]:
feature_weight = {
    'power steering': 1,
    'power windows front': 1, 
    'air conditioner': 1,
    'heater': 1, 
    'adjustable head lights': 2,
    'manually adjustable exterior rear view mirror': 1, 
    'centeral locking': 2,
    'child safety locks': 2, 
    'number of speaker': 1,
    'digital odometer': 1, 
    'electronic multi tripmeter': 1,
    'fabric upholstery':1, 
    'glove compartment':1,
    'dual tone dashboard': 1,
    'adjustable steering': 1,
    'fabric upholstery': 1, 
    'digital clock': 1,
    'power windows rear': 1, 
    'remote trunk opener':1, 
    'remote fuel lid opener': 1,
    'low fuel warning light': 1,
    'accessory power outlet':1, 
    'vanity mirror':1, 
    'rear seat headrest':1,
    'cup holders front':1
}
import ast 
def computeScore(values):
    score = 0
    org =  ast.literal_eval(values)
    for index, val in enumerate(org):
        if val in feature_weight:
           score += feature_weight.get(val, 0)
    return score

				


df['feature_score'] = df['comfort_features'].apply(computeScore) +  df['interior_features'].apply(computeScore) + \
df['exterior_features'].apply(computeScore) + +  df['safety_features'].apply(computeScore)

In [119]:
df['feature_score'].head()
df.drop(columns=['comfort_features', 'interior_features', 'exterior_features', 'safety_features' ], inplace=True)

In [121]:
df.info()
# saving to csv
# df.to_csv("cleaned_cars_dekho.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37813 entries, 0 to 37812
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   myear                 37813 non-null  int64  
 1   body                  37813 non-null  object 
 2   transmission          37813 non-null  object 
 3   fuel                  37813 non-null  object 
 4   km                    37813 non-null  float64
 5   ip                    37813 non-null  int64  
 6   threesixty            37813 non-null  bool   
 7   oem                   37813 non-null  object 
 8   model                 37813 non-null  object 
 9   variant               37813 non-null  object 
 10  selling_price         37813 non-null  float64
 11  utype                 37813 non-null  object 
 12  carType               37813 non-null  object 
 13  color                 37813 non-null  object 
 14  engine_type           37813 non-null  object 
 15  no_of_cylinder     

In [123]:
# List of columns to convert
numeric_columns = [
    'no_of_cylinder', 'valves_per_cylinder', 'wheel_base', 'front_tread', 'rear_tread',
    'kerb_weight', 'gross_weight', 'seats', 'top_speed', 'doors', 'cargo_volume'
]

# Convert columns to numeric (handling errors safely)
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

float_column = ['turning_radius','acceleration','max_power_delivered','max_torque_delivered']
df[float_column] = df[float_column].apply(pd.to_numeric, errors='coerce')

print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37813 entries, 0 to 37812
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   myear                 37813 non-null  int64  
 1   body                  37813 non-null  object 
 2   transmission          37813 non-null  object 
 3   fuel                  37813 non-null  object 
 4   km                    37813 non-null  float64
 5   ip                    37813 non-null  int64  
 6   threesixty            37813 non-null  bool   
 7   oem                   37813 non-null  object 
 8   model                 37813 non-null  object 
 9   variant               37813 non-null  object 
 10  selling_price         37813 non-null  float64
 11  utype                 37813 non-null  object 
 12  carType               37813 non-null  object 
 13  color                 37813 non-null  object 
 14  engine_type           37813 non-null  object 
 15  no_of_cylinder     

In [137]:
string_columns = [
    'body', 'transmission', 'fuel', 'oem', 'model', 'variant', 'utype', 'carType', 
    'color', 'engine_type', 'steering_type', 'front_brake_type', 'rear_brake_type', 
    'tyre_type', 'model_type_new', 'exterior_color', 'owner_type'
]


# df[string_columns] = df[string_columns].astype("string")
df[string_columns] = df[string_columns].astype(str)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37813 entries, 0 to 37812
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   myear                 37813 non-null  int64  
 1   body                  37813 non-null  object 
 2   transmission          37813 non-null  object 
 3   fuel                  37813 non-null  object 
 4   km                    37813 non-null  float64
 5   ip                    37813 non-null  int64  
 6   threesixty            37813 non-null  bool   
 7   oem                   37813 non-null  object 
 8   model                 37813 non-null  object 
 9   variant               37813 non-null  object 
 10  selling_price         37813 non-null  float64
 11  utype                 37813 non-null  object 
 12  carType               37813 non-null  object 
 13  color                 37813 non-null  object 
 14  engine_type           37813 non-null  object 
 15  no_of_cylinder     

In [139]:
# saving to csv
df.to_csv("cleaned_cars_dekho.csv", index=False)

In [152]:
# catrgorical variables
# Create Column Transformer with 3 types of transformers
numerical_features = df.select_dtypes(exclude="object").columns
categorical_features = df.select_dtypes(include="object").columns

print(numeric_features)
print(categorical_features)

['myear', 'km', 'ip', 'threesixty', 'selling_price', 'no_of_cylinder', 'valves_per_cylinder', 'turbo_charger', 'super_charger', 'wheel_base', 'front_tread', 'rear_tread', 'kerb_weight', 'gross_weight', 'seats', 'turning_radius', 'top_speed', 'acceleration', 'doors', 'cargo_volume', 'max_power_delivered', 'max_torque_delivered', 'feature_score']
Index(['body', 'transmission', 'fuel', 'oem', 'model', 'variant', 'utype',
       'carType', 'color', 'engine_type', 'gear_box', 'steering_type',
       'front_brake_type', 'rear_brake_type', 'tyre_type', 'model_type_new',
       'exterior_color', 'owner_type'],
      dtype='object')
